In [1]:
import cobra
from cobra.io import read_sbml_model
from cobra import Reaction, Model, Gene, Metabolite
from cobra.flux_analysis import find_blocked_reactions

In [2]:
Syn6803_file = 'C:\\Users\\path to file\\1_iSynCJ816_WT.xml'
Syn6803 = read_sbml_model(Syn6803_file)

No objective coefficients in model. Unclear what should be optimized


In [3]:
Syn6803.objective = Syn6803.reactions.get_by_id('BIOMASS_Ec_SynAuto_1')

In [6]:
for r in Syn6803.exchanges:
    print(r.name, r.bounds)

Acetate exchange (0.0, 999999.0)
2-Oxoglutarate exchange (0.0, 999999.0)
L-Alanine exchange (0.0, 999999.0)
Calcium exchange (-999999.0, 999999.0)
Citrate exchange (0.0, 999999.0)
EX co LPAREN e RPAREN  (0.0, 999999.0)
CO2 exchange (0.0, 999999.0)
Co2+ exchange (-999999.0, 999999.0)
Cu2+ exchange (-999999.0, 999999.0)
Cyanate exchange (0.0, 999999.0)
Fe2+ exchange (-999999.0, 999999.0)
Fe3+ exchange (-999999.0, 999999.0)
D-Fructose exchange (0.0, 999999.0)
Fumarate exchange (0.0, 999999.0)
D-Glucose exchange (-0.85, 999999.0)
Glcglyc exchange (0.0, 999999.0)
L-Glutamine exchange (0.0, 999999.0)
L-Glutamate exchange (0.0, 999999.0)
Glycine exchange (0.0, 999999.0)
H+ exchange (-999999.0, 999999.0)
H2 exchange (0.0, 999999.0)
H2O exchange (-999999.0, 999999.0)
EX hco3 LPAREN e RPAREN  (0.0, 999999.0)
L-Histidine exchange (0.0, 999999.0)
K+ exchange (-999999.0, 999999.0)
L-Leucine exchange (0.0, 999999.0)
L-Lysine exchange (0.0, 999999.0)
L-Malate exchange (0.0, 999999.0)
Mg exchange (-99

# Fixing RBPC

In [4]:
H2CO3 = Syn6803.reactions.HCO3E1
Syn6803.reactions.HCO3E1.add_metabolites({Syn6803.metabolites.get_by_id('h2co3_x'): 1.0})
Syn6803.reactions.HCO3E1.reaction

'co2_x + h2o_x <=> h2co3_x'

In [5]:
Syn6803.reactions.HCO3E1

Reaction identifier,HCO3E1
Name,HCO3 equilibration reaction
Memory address,0x257f4904650
Stoichiometry,co2_x + h2o_x <=> h2co3_x CO2 CO2 + H2O H2O <=> Carbonic acid
GPR,SGL_RS13835 or SGL_RS10110
Lower bound,-999999.0
Upper bound,999999.0


# Adding HCO3 <-> CO2 equillibrium

In [8]:
internal_HCO3_CO2_eqillibrium = Reaction('HCO3E2')
internal_HCO3_CO2_eqillibrium.name = 'HCO3 equilibration reaction (Carboxysome)'
internal_HCO3_CO2_eqillibrium.id = 'HCO3E2'
internal_HCO3_CO2_eqillibrium.subsystem = 'cytosol'
internal_HCO3_CO2_eqillibrium.lower_bound = -0.  # This is the default
internal_HCO3_CO2_eqillibrium.upper_bound = 0.  # This is the default
internal_HCO3_CO2_eqillibrium.add_metabolites({Syn6803.metabolites.get_by_id('hco3_c'): -1.0,
                                               Syn6803.metabolites.get_by_id('h_c'): -1.0,
                          Syn6803.metabolites.get_by_id('h2o_c'): 1.0,
                          Syn6803.metabolites.get_by_id('co2_c'): 1.0})
Syn6803.add_reactions([internal_HCO3_CO2_eqillibrium])

# Adding Ethanol Pathway

In [7]:
etoh_e = Metabolite(
    'etoh_e',
    name='Ethanol',
    compartment='e')
Syn6803.add_metabolites(etoh_e)


PDC_reaction = Reaction('PYRDC')
PDC_reaction.name = 'Pyruvate decarboxylase'
PDC_reaction.id = 'PYRDC'
PDC_reaction.gene_reaction_rule = 'PDC_ZYMMO'
PDC_reaction.subsystem = 'cytosol'
PDC_reaction.lower_bound = 0
PDC_reaction.upper_bound = 0
PDC_reaction.add_metabolites({Syn6803.metabolites.get_by_id('pyr_c'): -1.0,
                              Syn6803.metabolites.get_by_id('h_c'): -1.0,
                          Syn6803.metabolites.get_by_id('acald_c'): 1.0,
                          Syn6803.metabolites.get_by_id('co2_c'): 1.0})

Etoh_objective = Reaction('ETOHt')
Etoh_objective.name = 'Ethanol reversible transport'
Etoh_objective.id = 'ETOHt'
Etoh_objective.subsystem = 'cytosol'
Etoh_objective.lower_bound = 0.  # This is the default
Etoh_objective.upper_bound = 0.  # This is the default
Etoh_objective.add_metabolites({Syn6803.metabolites.get_by_id('etoh_c'): -1.0,
                          Syn6803.metabolites.get_by_id('etoh_e'): 1.0})

Syn6803.add_reaction(Etoh_objective)
Syn6803.add_reaction(PDC_reaction)
Syn6803.add_boundary(Syn6803.metabolites.get_by_id("etoh_e"), type="exchange")
Syn6803.reactions.EX_etoh_e.bounds = (0.0, 999999)

In [8]:
def Ethanol_On(Model):
    Model.reactions.PYRDC.bounds = (0,999999)
    Model.reactions.ETOHt.bounds = (0,999999)
    return

def Ethanol_Off(Model):
    Model.reactions.PYRDC.bounds = (0,0)
    Model.reactions.ETOHt.bounds = (0,0)
    return

# Adding Lactate Pathway

In [9]:
lac__L_c = Metabolite(
    'lac__L_c',
    name='L-Lactate',
    compartment='c')

lac__L_e = Metabolite(
    'lac__L_e',
    name='L-Lactate',
    compartment='e')

Syn6803.add_metabolites(lac__L_c)
Syn6803.add_metabolites(lac__L_e)
    
LDH_reaction = Reaction('LDH_L_NADPH')
LDH_reaction.name = 'L-lactate dehydrogenase NADPH'
LDH_reaction.id = 'LDH_L_NADPH'
LDH_reaction.gene_reaction_rule = 'LDH2_LACLA'
LDH_reaction.subsystem = 'cytosol'
LDH_reaction.lower_bound = 0
LDH_reaction.upper_bound = 0
LDH_reaction.add_metabolites({Syn6803.metabolites.get_by_id('pyr_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadph_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadp_c'): 1.0,
                        Syn6803.metabolites.get_by_id('lac__L_c'): 1.0})

LDH_reaction2 = Reaction('LDH_L')
LDH_reaction2.name = 'L-lactate dehydrogenase NADH'
LDH_reaction2.id = 'LDH_L'
LDH_reaction2.gene_reaction_rule = 'LDH2_LACLA'
LDH_reaction2.subsystem = 'cytosol'
LDH_reaction2.lower_bound = 0
LDH_reaction2.upper_bound = 0
LDH_reaction2.add_metabolites({Syn6803.metabolites.get_by_id('pyr_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadh_c'): -1.0,
                        Syn6803.metabolites.get_by_id('h_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nad_c'): 1.0,
                        Syn6803.metabolites.get_by_id('lac__L_c'): 1.0})

Lac_Secretion = Reaction('LACLt')
Lac_Secretion.name = 'L-Lactate secretion'
Lac_Secretion.id = 'LACLt'
Lac_Secretion.subsystem = 'cytosol'
Lac_Secretion.lower_bound = 0.  # This is the default
Lac_Secretion.upper_bound = 0.  # This is the default
Lac_Secretion.add_metabolites({Syn6803.metabolites.get_by_id('lac__L_c'): -1.0,
                          Syn6803.metabolites.get_by_id('lac__L_e'): 1.0})


Syn6803.add_reaction(LDH_reaction)
Syn6803.add_reaction(LDH_reaction2)
Syn6803.add_reaction(Lac_Secretion)
Syn6803.add_boundary(Syn6803.metabolites.get_by_id("lac__L_e"), type="exchange")
Syn6803.reactions.EX_lac__L_e.bounds = (0.0, 999999)

In [10]:
def Lactate_On(Model):
    Model.reactions.LDH_L_NADPH.bounds = (0,999999)
    Model.reactions.LDH_L.bounds = (0,999999)
    Model.reactions.LACLt.bounds = (0,999999)
    return

def Lactate_Off(Model):
    Model.reactions.LDH_L_NADPH.bounds = (0,0)
    Model.reactions.LDH_L.bounds = (0,0)
    Model.reactions.LACLt.bounds = (0,0)
    return

# Ethylene Pathway

In [11]:
Ethylene_c = Metabolite(
    'ethe_c',
    name='ethylene',
    compartment='c')

Ethylene_e = Metabolite(
    'ethe_e',
    name='ethylene',
    compartment='e')

Guanidine_c = Metabolite(
    'M02035_c',
    name='guanidine',
    compartment='c')

Guanidine_e = Metabolite(
    'M02035_e',
    name='guanidine',
    compartment='e')

Syn6803.add_metabolites(Ethylene_c)
Syn6803.add_metabolites(Ethylene_e)
Syn6803.add_metabolites(Guanidine_c)
Syn6803.add_metabolites(Guanidine_e)


EFE_reaction = Reaction('EFE')
EFE_reaction.name = 'Ethylene Forming Enzyme Grouped Reaction'
EFE_reaction.id = 'EFE'
EFE_reaction.gene_reaction_rule = 'EFE_PSESH'
EFE_reaction.subsystem = 'cytosol'
EFE_reaction.lower_bound = 0
EFE_reaction.upper_bound = 0
EFE_reaction.add_metabolites({Syn6803.metabolites.get_by_id('akg_c'): -2.0,
                              Syn6803.metabolites.get_by_id('arg__L_c'): -1.0,
                              Syn6803.metabolites.get_by_id('o2_c'): -3.0,
                        Syn6803.metabolites.get_by_id('ethe_e'): 2.0,
                        Syn6803.metabolites.get_by_id('succ_c'): 1.0,
                        Syn6803.metabolites.get_by_id('h2o_c'): 3.0,
                        Syn6803.metabolites.get_by_id('co2_c'): 7.0,
                        Syn6803.metabolites.get_by_id('M02035_c'): 1.0,
                        Syn6803.metabolites.get_by_id('1pyr5c_c'): 1.0})

Ethylene_Secretion = Reaction('EFEt')
Ethylene_Secretion.name = 'Ethylene secretion'
Ethylene_Secretion.id = 'EFEt'
Ethylene_Secretion.subsystem = 'cytosol'
Ethylene_Secretion.lower_bound = 0.  # This is the default
Ethylene_Secretion.upper_bound = 0.  # This is the default
Ethylene_Secretion.add_metabolites({Syn6803.metabolites.get_by_id('ethe_c'): -1.0,
                          Syn6803.metabolites.get_by_id('ethe_e'): 1.0})

Guanidine_Secretion = Reaction('M02035t')
Guanidine_Secretion.name = 'Guanidine secretion'
Guanidine_Secretion.id = 'M02035t'
Guanidine_Secretion.subsystem = 'cytosol'
Guanidine_Secretion.lower_bound = 0.  # This is the default
Guanidine_Secretion.upper_bound = 0.  # This is the default
Guanidine_Secretion.add_metabolites({Syn6803.metabolites.get_by_id('M02035_c'): -1.0,
                          Syn6803.metabolites.get_by_id('M02035_e'): 1.0})



Syn6803.add_reaction(EFE_reaction)
Syn6803.add_reaction(Ethylene_Secretion)
Syn6803.add_reaction(Guanidine_Secretion)
Syn6803.add_boundary(Syn6803.metabolites.get_by_id("M02035_e"), type="exchange")
Syn6803.reactions.EX_M02035_e.bounds = (0.0, 999999)
Syn6803.add_boundary(Syn6803.metabolites.get_by_id("ethe_e"), type="exchange")
Syn6803.reactions.EX_ethe_e.bounds = (0.0, 999999)

In [12]:
def Ethylene_On(Model):
    Model.reactions.EFE.bounds = (0,999999)
    Model.reactions.EFEt.bounds = (0,999999)
    Model.reactions.M02035t.bounds = (0,999999)
    return

def Ethylene_Off(Model):
    Model.reactions.EFE.bounds = (0,0)
    Model.reactions.EFEt.bounds = (0,0)
    Model.reactions.M02035t.bounds = (0,0)
    return

# Isobutanol Pathway

In [13]:
Isobutyralderhyde_c = Metabolite(
    '2mppal_c',
    name='2 methylpropanal (isobutyraldehyde)',
    compartment='c')

Isobutanol_e = Metabolite(
    '2mpa_e',
    name='2mpa',
    compartment='e')

Isobutanol_c = Metabolite(
    '2mpa_c',
    name='2mpa',
    compartment='c')

Syn6803.add_metabolites(Isobutyralderhyde_c)
Syn6803.add_metabolites(Isobutanol_e)
Syn6803.add_metabolites(Isobutanol_c)


KIVD_reaction = Reaction('3MOBDC')
KIVD_reaction.name = 'Alpha-ketoisovalerate (3 Methyl 2 oxobutanoate) decarboxylase (KIVD)'
KIVD_reaction.id = '3MOBDC'
KIVD_reaction.gene_reaction_rule = 'D2BR82_LACLK'
KIVD_reaction.subsystem = 'cytosol'
KIVD_reaction.lower_bound = 0
KIVD_reaction.upper_bound = 0
KIVD_reaction.add_metabolites({Syn6803.metabolites.get_by_id('3mob_c'): -1.0,
                               Syn6803.metabolites.get_by_id('h_c'): -1.0,
                        Syn6803.metabolites.get_by_id('2mppal_c'): 1.0,
                        Syn6803.metabolites.get_by_id('co2_c'): 1.0})


isobutyraldehyde_reduction1 = Reaction('isobutyraldehyde_reduction1')
isobutyraldehyde_reduction1.name = 'isobutyraldehyde reduction NADPH'
isobutyraldehyde_reduction1.id = 'IBDH_NADPH'
isobutyraldehyde_reduction1.gene_reaction_rule = 'IBDH'
isobutyraldehyde_reduction1.subsystem = 'cytosol'
isobutyraldehyde_reduction1.lower_bound = 0
isobutyraldehyde_reduction1.upper_bound = 0
isobutyraldehyde_reduction1.add_metabolites({Syn6803.metabolites.get_by_id('2mppal_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadph_c'): -1.0,
                        Syn6803.metabolites.get_by_id('h_c'): -1.0,
                        Syn6803.metabolites.get_by_id('2mpa_c'): 1.0,
                        Syn6803.metabolites.get_by_id('nadp_c'): 1.0})

isobutyraldehyde_reduction2 = Reaction('isobutyraldehyde_reduction2')
isobutyraldehyde_reduction2.name = 'isobutyraldehyde reduction NADH'
isobutyraldehyde_reduction2.id = 'IBDH_NADH'
isobutyraldehyde_reduction2.gene_reaction_rule = 'IBDH'
isobutyraldehyde_reduction2.subsystem = 'cytosol'
isobutyraldehyde_reduction2.lower_bound = 0
isobutyraldehyde_reduction2.upper_bound = 0
isobutyraldehyde_reduction2.add_metabolites({Syn6803.metabolites.get_by_id('2mppal_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadh_c'): -1.0,
                        Syn6803.metabolites.get_by_id('h_c'): -1.0,
                        Syn6803.metabolites.get_by_id('2mpa_c'): 1.0,
                        Syn6803.metabolites.get_by_id('nad_c'): 1.0})

Isobutanol_Secretion = Reaction('2MPPALt')
Isobutanol_Secretion.name = '2mpa_c (Isobutanol) transport by diffusion'
Isobutanol_Secretion.id = '2MPPALt'
Isobutanol_Secretion.subsystem = 'cytosol'
Isobutanol_Secretion.lower_bound = 0.  # This is the default
Isobutanol_Secretion.upper_bound = 999999.  # This is the default
Isobutanol_Secretion.add_metabolites({Syn6803.metabolites.get_by_id('2mpa_c'): -1.0,
                          Syn6803.metabolites.get_by_id('2mpa_e'): 1.0})

Syn6803.add_reaction(KIVD_reaction)
Syn6803.add_reaction(isobutyraldehyde_reduction1)
Syn6803.add_reaction(isobutyraldehyde_reduction2)
Syn6803.add_reaction(Isobutanol_Secretion)
Syn6803.add_boundary(Syn6803.metabolites.get_by_id("2mpa_e"), type="exchange")
Syn6803.reactions.EX_2mpa_e.bounds = (0.0, 999999)

In [14]:
def Isobutanol_On(Model):
    Model.reactions.get_by_id('3MOBDC').bounds = (0,999999)
    Model.reactions.IBDH_NADPH.bounds = (0,999999)
    Model.reactions.IBDH_NADH.bounds = (0,999999)
    Model.reactions.get_by_id('2MPPALt').bounds = (0,999999)    
    return

def Isobutanol_Off(Model):
    Model.reactions.get_by_id('3MOBDC').bounds = (0,0)
    Model.reactions.IBDH_NADPH.bounds = (0,0)
    Model.reactions.IBDH_NADH.bounds = (0,0)
    Model.reactions.get_by_id('2MPPALt').bounds = (0,0)    
    return

# 1-butanol pathway

In [15]:
Crotonyl_CoA_c = Metabolite(
    'ctncoa_c',
    name='Crotonyl-CoA',
    compartment='c')

Butyryl_CoA_c = Metabolite(
    'btcoa_c',
    name='Butanoyl-CoA (butyryl-CoA)',
    compartment='c')

Butyraldehyde_c = Metabolite(
    'btal_c',
    name='Butyraldehyde',
    compartment='c')

Butanol_c = Metabolite(
    'btoh_c',
    name='Butanol',
    compartment='c')

Butanol_e = Metabolite(
    'btoh_e',
    name='Butanol',
    compartment='e')

Syn6803.add_metabolites(Crotonyl_CoA_c)
Syn6803.add_metabolites(Butyryl_CoA_c)
Syn6803.add_metabolites(Butyraldehyde_c)
Syn6803.add_metabolites(Butanol_c)
Syn6803.add_metabolites(Butanol_e)


HBD_CLOAB_reaction = Reaction('HBD')
HBD_CLOAB_reaction.name = '3-hydroxybutyryl-CoA dehydrogenase'
HBD_CLOAB_reaction.id = 'HBD'
HBD_CLOAB_reaction.gene_reaction_rule = 'HBD_CLOAB'
HBD_CLOAB_reaction.subsystem = 'cytosol'
HBD_CLOAB_reaction.lower_bound = 0
HBD_CLOAB_reaction.upper_bound = 0
HBD_CLOAB_reaction.add_metabolites({Syn6803.metabolites.get_by_id('aacoa_c'): -1.0,
                        Syn6803.metabolites.get_by_id('h_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadh_c'): -1.0,
                        Syn6803.metabolites.get_by_id('3hbcoa__R_c'): 1.0,
                        Syn6803.metabolites.get_by_id('nad_c'): 1.0})


CRT_CLOAB_reaction = Reaction('CRT')
CRT_CLOAB_reaction.name = 'Crotonase'
CRT_CLOAB_reaction.id = 'CRT'
CRT_CLOAB_reaction.gene_reaction_rule = 'CRT_CLOAB'
CRT_CLOAB_reaction.subsystem = 'cytosol'
CRT_CLOAB_reaction.lower_bound = 0
CRT_CLOAB_reaction.upper_bound = 0
CRT_CLOAB_reaction.add_metabolites({Syn6803.metabolites.get_by_id('3hbcoa__R_c'): -1.0,
                        Syn6803.metabolites.get_by_id('ctncoa_c'): 1.0,
                        Syn6803.metabolites.get_by_id('h2o_c'): 1.0})

FABV_TREDE_reaction = Reaction('FABV_reaction')
FABV_TREDE_reaction.name = 'Trans-2-enoyl-CoA reductase'
FABV_TREDE_reaction.id = 'TER'
FABV_TREDE_reaction.gene_reaction_rule = 'FABV_TREDE'
FABV_TREDE_reaction.subsystem = 'cytosol'
FABV_TREDE_reaction.lower_bound = 0
FABV_TREDE_reaction.upper_bound = 0
FABV_TREDE_reaction.add_metabolites({Syn6803.metabolites.get_by_id('ctncoa_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadh_c'): -1.0,
                        Syn6803.metabolites.get_by_id('h_c'): -1.0,
                        Syn6803.metabolites.get_by_id('btcoa_c'): 1.0,
                        Syn6803.metabolites.get_by_id('nad_c'): 1.0})

AdhE2_reaction1 = Reaction('BTCOARx')
AdhE2_reaction1.name = 'Butyryl-coA reductase (NADH)'
AdhE2_reaction1.id = 'BTCOARx'
AdhE2_reaction1.gene_reaction_rule = 'Q9ANR5_CLOAT'
AdhE2_reaction1.subsystem = 'cytosol'
AdhE2_reaction1.lower_bound = 0
AdhE2_reaction1.upper_bound = 0
AdhE2_reaction1.add_metabolites({Syn6803.metabolites.get_by_id('btcoa_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadh_c'): -1.0,
                        Syn6803.metabolites.get_by_id('h_c'): -1.0,
                        Syn6803.metabolites.get_by_id('coa_c'): 1.0,         
                        Syn6803.metabolites.get_by_id('btal_c'): 1.0,
                        Syn6803.metabolites.get_by_id('nad_c'): 1.0})

AdhE2_reaction2 = Reaction('BTS')
AdhE2_reaction2.name = 'Butanol synthase (Aldehyde-alcohol dehydrogenase)'
AdhE2_reaction2.id = 'BTS'
AdhE2_reaction2.gene_reaction_rule = 'Q9ANR5_CLOAT'
AdhE2_reaction2.subsystem = 'cytosol'
AdhE2_reaction2.lower_bound = 0
AdhE2_reaction2.upper_bound = 0
AdhE2_reaction2.add_metabolites({Syn6803.metabolites.get_by_id('btal_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadh_c'): -1.0,
                        Syn6803.metabolites.get_by_id('h_c'): -1.0,
                        Syn6803.metabolites.get_by_id('coa_c'): 1.0,         
                        Syn6803.metabolites.get_by_id('btoh_c'): 1.0,
                        Syn6803.metabolites.get_by_id('nad_c'): 1.0})

Butan_1_ol_Secretion = Reaction('BTOHt')
Butan_1_ol_Secretion.name = 'Butanol transport by diffusion'
Butan_1_ol_Secretion.id = 'BTOHt'
Butan_1_ol_Secretion.subsystem = 'cytosol'
Butan_1_ol_Secretion.lower_bound = 0.  # This is the default
Butan_1_ol_Secretion.upper_bound = 0.  # This is the default
Butan_1_ol_Secretion.add_metabolites({Syn6803.metabolites.get_by_id('btoh_c'): -1.0,
                          Syn6803.metabolites.get_by_id('btoh_e'): 1.0})

Syn6803.add_reaction(HBD_CLOAB_reaction)
Syn6803.add_reaction(CRT_CLOAB_reaction)
Syn6803.add_reaction(FABV_TREDE_reaction)
Syn6803.add_reaction(AdhE2_reaction1)
Syn6803.add_reaction(AdhE2_reaction2)
Syn6803.add_reaction(Butan_1_ol_Secretion)
Syn6803.add_boundary(Syn6803.metabolites.get_by_id("btoh_e"), type="exchange")
Syn6803.reactions.EX_btoh_e.bounds = (0.0, 999999)

In [29]:
def Butan_1_ol_On(Model):
    Model.reactions.HBD.bounds = (0,999999)
    Model.reactions.CRT.bounds = (0,999999)
    Model.reactions.TER.bounds = (0,999999)
    Model.reactions.BTCOARx.bounds = (0,999999)
    Model.reactions.BTS.bounds = (0,999999)
    Model.reactions.BTOHt.bounds = (0,999999)  
    return

def Butan_1_ol_Off(Model):
    Model.reactions.HBD.bounds = (0,0)
    Model.reactions.CRT.bounds = (0,0)
    Model.reactions.TER.bounds = (0,0)
    Model.reactions.BTCOARx.bounds = (0,0)
    Model.reactions.BTS.bounds = (0,0)
    Model.reactions.BTOHt.bounds = (0,0)  
    return

# Farnescene Pathway

In [17]:
farnesene_c = Metabolite(
    'farnesene_c',
    name='farnesene',
    compartment='c')

farnesene_e = Metabolite(
    'farnesene_e',
    name='farnesene',
    compartment='e')

Syn6803.add_metabolites(farnesene_c)
Syn6803.add_metabolites(farnesene_e)

Farnesene_synthase_reaction = Reaction('FARNESENEs')
Farnesene_synthase_reaction.name = 'Alpha-Farnesene synthase reaction'
Farnesene_synthase_reaction.id = 'FARNESENEs'
Farnesene_synthase_reaction.gene_reaction_rule = 'Q675K8_PICAB'
Farnesene_synthase_reaction.subsystem = 'cytosol'
Farnesene_synthase_reaction.lower_bound = 0.  # This is the default
Farnesene_synthase_reaction.upper_bound = 0.  # This is the default
Farnesene_synthase_reaction.add_metabolites({Syn6803.metabolites.get_by_id('frdp_c'): -1.0,
                          Syn6803.metabolites.get_by_id('farnesene_c'): 1.0})

Farnesene_Secretion = Reaction('FARNESENEt')
Farnesene_Secretion.name = 'Farnesene transport by diffusion'
Farnesene_Secretion.id = 'FARNESENEt'
Farnesene_Secretion.subsystem = 'cytosol'
Farnesene_Secretion.lower_bound = 0.  # This is the default
Farnesene_Secretion.upper_bound = 0.  # This is the default
Farnesene_Secretion.add_metabolites({Syn6803.metabolites.get_by_id('farnesene_c'): -1.0,
                          Syn6803.metabolites.get_by_id('farnesene_e'): 1.0})

Syn6803.add_reaction(Farnesene_synthase_reaction)
Syn6803.add_reaction(Farnesene_Secretion)
Syn6803.add_boundary(Syn6803.metabolites.get_by_id("farnesene_e"), type="exchange")
Syn6803.reactions.EX_farnesene_e.bounds = (0.0, 999999)

In [18]:
def Farnesene_On(Model):
    Model.reactions.FARNESENEs.bounds = (0,999999)
    Model.reactions.FARNESENEt.bounds = (0,999999)
    return

def Farnesene_Off(Model):
    Model.reactions.FARNESENEs.bounds = (0,0)
    Model.reactions.FARNESENEt.bounds = (0,0)
    return

# Limonene Synthase Pathway

In [19]:
limonene_c = Metabolite(
    'limnen_c',
    name='limonene',
    compartment='c')

limonene_e = Metabolite(
    'limnen_e',
    name='limonene',
    compartment='e')

Syn6803.add_metabolites(limonene_c)
Syn6803.add_metabolites(limonene_e)

Limonene_synthase_reaction = Reaction('LIMONENEs')
Limonene_synthase_reaction.name = 'Beta-Limonene synthase reaction'
Limonene_synthase_reaction.id = 'LIMONENEs'
Limonene_synthase_reaction.gene_reaction_rule = 'TPSBF_MENPI'
Limonene_synthase_reaction.subsystem = 'cytosol'
Limonene_synthase_reaction.lower_bound = 0.  # This is the default
Limonene_synthase_reaction.upper_bound = 0.  # This is the default
Limonene_synthase_reaction.add_metabolites({Syn6803.metabolites.get_by_id('grdp_c'): -1.0,
                          Syn6803.metabolites.get_by_id('limnen_c'): 1.0})

Limonene_Secretion = Reaction('LIMNENt')
Limonene_Secretion.name = 'limonene transport by diffusion'
Limonene_Secretion.id = 'LIMNENt'
Limonene_Secretion.subsystem = 'cytosol'
Limonene_Secretion.lower_bound = 0.  # This is the default
Limonene_Secretion.upper_bound = 0.  # This is the default
Limonene_Secretion.add_metabolites({Syn6803.metabolites.get_by_id('limnen_c'): -1.0,
                          Syn6803.metabolites.get_by_id('limnen_e'): 1.0})

Syn6803.add_reaction(Limonene_synthase_reaction)
Syn6803.add_reaction(Limonene_Secretion)
Syn6803.add_boundary(Syn6803.metabolites.get_by_id("limnen_e"), type="exchange")
Syn6803.reactions.EX_limnen_e.bounds = (0.0, 999999)

In [20]:
def Limonene_On(Model):
    Model.reactions.LIMONENEs.bounds = (0,999999)
    Model.reactions.LIMNENt.bounds = (0,999999)
    return

def Limonene_Off(Model):
    Model.reactions.LIMONENEs.bounds = (0,0)
    Model.reactions.LIMNENt.bounds = (0,0)
    return

# 2,3 Butandiol

In [21]:
Butanediol_c = Metabolite(
    '23btdl_c',
    name='Butane-2,3-diol',
    compartment='c')

Butanediol_e = Metabolite(
    '23btdl_e',
    name='Butane-2,3-diol',
    compartment='e')



Syn6803.add_metabolites(Butanediol_c)
Syn6803.add_metabolites(Butanediol_e)

Butandiol_reaction0 = Reaction('ACLS')
Butandiol_reaction0.name = 'Acetolactate synthase'
Butandiol_reaction0.id = 'ACLS'
Butandiol_reaction0.gene_reaction_rule = 'V7ZSM0_ENTFL'
Butandiol_reaction0.subsystem = 'cytosol'
Butandiol_reaction0.lower_bound = 0.  # This is the default
Butandiol_reaction0.upper_bound = 0.  # This is the default
Butandiol_reaction0.add_metabolites({Syn6803.metabolites.get_by_id('pyr_c'): -2.0,
                        Syn6803.metabolites.get_by_id('h_c'): -1.0,        
                        Syn6803.metabolites.get_by_id('R_actn_c'): 1.0,
                        Syn6803.metabolites.get_by_id('co2_c'): 1.0})

Butandiol_reaction = Reaction('ACLDC')
Butandiol_reaction.name = 'Acetolactate decarboxylase'
Butandiol_reaction.id = 'ACLDC'
Butandiol_reaction.gene_reaction_rule = 'ALDC_LACLA'
Butandiol_reaction.subsystem = 'cytosol'
Butandiol_reaction.lower_bound = 0.  # This is the default
Butandiol_reaction.upper_bound = 0.  # This is the default
Butandiol_reaction.add_metabolites({Syn6803.metabolites.get_by_id('R_actn_c'): -1.0,
                        Syn6803.metabolites.get_by_id('co2_c'): -1.0,        
                        Syn6803.metabolites.get_by_id('alac__S_c'): 1.0,
                        Syn6803.metabolites.get_by_id('h_c'): 1.0})


Butandiol_reaction2 = Reaction('BTDD_RR')
Butandiol_reaction2.name = 'R R butanediol dehydrogenase'
Butandiol_reaction2.id = 'BTDD_RR'
Butandiol_reaction2.gene_reaction_rule = 'A0A1V0NF97_LACLL'
Butandiol_reaction2.subsystem = 'cytosol'
Butandiol_reaction2.lower_bound = 0.  # This is the default
Butandiol_reaction2.upper_bound = 0.  # This is the default
Butandiol_reaction2.add_metabolites({Syn6803.metabolites.get_by_id('alac__S_c'): -1.0,
                        Syn6803.metabolites.get_by_id('nadh_c'): -1.0,
                        Syn6803.metabolites.get_by_id('h_c'): -1.0,        
                        Syn6803.metabolites.get_by_id('23btdl_c'): 1.0,
                        Syn6803.metabolites.get_by_id('nad_c'): 1.0})


Butandiol_reaction3 = Reaction('23BTDLt')
Butandiol_reaction3.name = '2,3-Butandiol transport by diffusion'
Butandiol_reaction3.id = '23BTDLt'
Butandiol_reaction3.subsystem = 'cytosol'
Butandiol_reaction3.lower_bound = 0.  # This is the default
Butandiol_reaction3.upper_bound = 0.  # This is the default
Butandiol_reaction3.add_metabolites({Syn6803.metabolites.get_by_id('23btdl_c'): -1.0,
                          Syn6803.metabolites.get_by_id('23btdl_e'): 1.0})

Syn6803.add_reaction(Butandiol_reaction0)
Syn6803.add_reaction(Butandiol_reaction)
Syn6803.add_reaction(Butandiol_reaction2)
Syn6803.add_reaction(Butandiol_reaction3)
Syn6803.add_boundary(Syn6803.metabolites.get_by_id("23btdl_e"), type="exchange")
Syn6803.reactions.EX_23btdl_e.bounds = (0.0, 999999)

In [22]:
def Butanediol_On(Model):
    Model.reactions.ACLS.bounds = (0,999999)
    Model.reactions.ACLDC.bounds = (0,999999)
    Model.reactions.BTDD_RR.bounds = (0,999999)
    Model.reactions.get_by_id("23BTDLt").bounds = (0,999999)
    return

def Butanediol_Off(Model):
    Model.reactions.ACLS.bounds = (0,0)
    Model.reactions.ACLDC.bounds = (0,0)
    Model.reactions.BTDD_RR.bounds = (0,999999)
    Model.reactions.get_by_id("23BTDLt").bounds = (0,999999)
    return

# Simulation

In [23]:
with Syn6803:
    print(Syn6803.summary())

Objective
1.0 BIOMASS_Ec_SynAuto_1 = 0.07653046659469571

Uptake
------
Metabolite     Reaction      Flux  C-Number C-Flux
     ca2_e     EX_ca2_e 0.0003453         0  0.00%
 cobalt2_e EX_cobalt2_e 0.0002489         0  0.00%
     cu2_e     EX_cu2_e 0.0002302         0  0.00%
     fe2_e     EX_fe2_e 0.0005729         0  0.00%
     fe3_e     EX_fe3_e 0.0005216         0  0.00%
  glc__D_e  EX_glc__D_e      0.85         0  0.00%
       h_e       EX_h_e     27.62         0  0.00%
       k_e       EX_k_e   0.01295         0  0.00%
     mg2_e     EX_mg2_e  0.002201         0  0.00%
     mn2_e     EX_mn2_e 0.0002318         0  0.00%
    mobd_e    EX_mobd_e 0.0002318         0  0.00%
     na1_e     EX_na1_e 0.0002898         0  0.00%
     no3_e     EX_no3_e    0.6899         0  0.00%
      o2_e      EX_o2_e     20.54         0  0.00%
      pi_e      EX_pi_e   0.05282         0  0.00%
     so4_e     EX_so4_e   0.01452         0  0.00%
     zn2_e     EX_zn2_e 0.0002302         0  0.00%

Secretion

In [25]:
Ethanol_On(Syn6803)
Lactate_On(Syn6803)
Ethylene_On(Syn6803)
Isobutanol_On(Syn6803)
Butan_1_ol_On(Syn6803)
Farnesene_On(Syn6803)
Limonene_On(Syn6803)
Butanediol_On(Syn6803)
    
with Syn6803:
    Syn6803.reactions.EX_glc__D_e.bounds = (0,0)
    Syn6803.reactions.EX_hco3_e.bounds = (-3.7, -3.7)
    Syn6803.reactions.EX_photon_e.bounds = (-999999, 0)
    Syn6803.reactions.NDH1_3u_1.bounds = (0,0)
    Syn6803.reactions.NDH1_4pp_1.bounds = (0,0)

Syn6803.summary(fva=0.95)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0003454,[0.0003281; 0.0003457],0,0.00%
cobalt2_e,EX_cobalt2_e,0.000249,[0.0002365; 0.0002492],0,0.00%
cu2_e,EX_cu2_e,0.0002303,[0.0002188; 0.0002305],0,0.00%
fe2_e,EX_fe2_e,0.000573,[0; 0.0005736],0,0.00%
fe3_e,EX_fe3_e,0.0005218,[0.0004957; 0.001096],0,0.00%
glc__D_e,EX_glc__D_e,0.85,[0.8075; 0.85],0,0.00%
h_e,EX_h_e,27.62,[20.84; 1E+06],0,0.00%
k_e,EX_k_e,0.01295,[0.0123; 0.01296],0,0.00%
mg2_e,EX_mg2_e,0.002202,[0.002092; 0.002204],0,0.00%
mn2_e,EX_mn2_e,0.0002319,[0.0002203; 0.0002321],0,0.00%


# Splitting PSI and PSII excitation

Reactions are already there as PHOTON680 and PHOTON700

# CO2 Reuptake

Labelling shows that 12% of fixed CO2 is lost - Evidence by MFA
With CO2 uptake set as 0, percentage lost is 13.15%, all evolved CO2 seems to be lost 

In [9]:
with Syn6803:
    Syn6803.reactions.EX_glc__D_e.bounds = (0,0)
    Syn6803.reactions.EX_hco3_e.bounds = (-3.7, -3.7)
    Syn6803.reactions.EX_photon_e.bounds = (-999999, 0)
    print(Syn6803.summary())

Objective
1.0 BIOMASS_Ec_SynAuto_1 = 0.08834483832944977

Uptake
------
Metabolite     Reaction      Flux  C-Number C-Flux
     ca2_e     EX_ca2_e 0.0003986         0  0.00%
 cobalt2_e EX_cobalt2_e 0.0002873         0  0.00%
     cu2_e     EX_cu2_e 0.0002657         0  0.00%
     fe2_e     EX_fe2_e 0.0006613         0  0.00%
     fe3_e     EX_fe3_e 0.0006022         0  0.00%
       h_e       EX_h_e     11.83         0  0.00%
    hco3_e    EX_hco3_e       3.7         0  0.00%
       k_e       EX_k_e   0.01495         0  0.00%
     mg2_e     EX_mg2_e  0.002541         0  0.00%
     mn2_e     EX_mn2_e 0.0002676         0  0.00%
    mobd_e    EX_mobd_e 0.0002676         0  0.00%
     na1_e     EX_na1_e 0.0003346         0  0.00%
     no3_e     EX_no3_e    0.7964         0  0.00%
  photon_e  EX_photon_e     46.47         0  0.00%
      pi_e      EX_pi_e   0.06098         0  0.00%
     so4_e     EX_so4_e   0.01676         0  0.00%
     zn2_e     EX_zn2_e 0.0002657         0  0.00%

Secretion

In [10]:
with Syn6803:
    Syn6803.reactions.EX_glc__D_e.bounds = (0,0)
    Syn6803.reactions.EX_hco3_e.bounds = (-3.7, -3.7)
    Syn6803.reactions.EX_photon_e.bounds = (-999999, 0)
    Syn6803.reactions.NDH1_3u_1.bounds = (0,0)
    Syn6803.reactions.NDH1_4pp_1.bounds = (0,0)
    print(Syn6803.summary())

Objective
1.0 BIOMASS_Ec_SynAuto_1 = 0.07672456099109423

Uptake
------
Metabolite     Reaction      Flux  C-Number C-Flux
     ca2_e     EX_ca2_e 0.0003462         0  0.00%
 cobalt2_e EX_cobalt2_e 0.0002495         0  0.00%
     cu2_e     EX_cu2_e 0.0002308         0  0.00%
     fe2_e     EX_fe2_e 0.0005743         0  0.00%
     fe3_e     EX_fe3_e  0.000523         0  0.00%
       h_e       EX_h_e     10.76         0  0.00%
    hco3_e    EX_hco3_e       3.7         0  0.00%
       k_e       EX_k_e   0.01298         0  0.00%
     mg2_e     EX_mg2_e  0.002207         0  0.00%
     mn2_e     EX_mn2_e 0.0002324         0  0.00%
    mobd_e    EX_mobd_e 0.0002324         0  0.00%
     na1_e     EX_na1_e 0.0002906         0  0.00%
     no3_e     EX_no3_e    0.6917         0  0.00%
  photon_e  EX_photon_e     40.36         0  0.00%
      pi_e      EX_pi_e   0.05296         0  0.00%
     so4_e     EX_so4_e   0.01456         0  0.00%
     zn2_e     EX_zn2_e 0.0002308         0  0.00%

Secretion

In [27]:
with Syn6803:
    Syn6803.reactions.EX_glc__D_e.bounds = (0,0)
    Syn6803.reactions.EX_hco3_e.bounds = (-3.7, -3.7)
    Syn6803.reactions.EX_photon_e.bounds = (-999999, 0)
    Syn6803.reactions.NDH1_3u_1.bounds = (0,0)
    Syn6803.reactions.NDH1_4pp_1.bounds = (0,0)
    Ethanol_On(Syn6803)
    Lactate_On(Syn6803)
    Ethylene_On(Syn6803)
    Isobutanol_On(Syn6803)
    Butan_1_ol_On(Syn6803)
    Farnesene_On(Syn6803)
    Limonene_On(Syn6803)
    Butanediol_On(Syn6803)
    print(Syn6803.summary(fva=0.95))

Objective
1.0 BIOMASS_Ec_SynAuto_1 = 0.0767428946428627

Uptake
------
Metabolite     Reaction      Flux                  Range  C-Number C-Flux
     ca2_e     EX_ca2_e 0.0003463  [0.000329; 0.0003467]         0  0.00%
 cobalt2_e EX_cobalt2_e 0.0002496 [0.0002371; 0.0002499]         0  0.00%
     cu2_e     EX_cu2_e 0.0002308 [0.0002193; 0.0002312]         0  0.00%
     fe2_e     EX_fe2_e 0.0005744         [0; 0.0005752]         0  0.00%
     fe3_e     EX_fe3_e 0.0005231  [0.0004969; 0.001099]         0  0.00%
       h_e       EX_h_e      10.7    [-8.333E+05; 1E+06]         0  0.00%
    hco3_e    EX_hco3_e       3.7             [3.7; 3.7]         0  0.00%
       k_e       EX_k_e   0.01298       [0.01234; 0.013]         0  0.00%
     mg2_e     EX_mg2_e  0.002207    [0.002097; 0.00221]         0  0.00%
     mn2_e     EX_mn2_e 0.0002325 [0.0002208; 0.0002328]         0  0.00%
    mobd_e    EX_mobd_e 0.0002325 [0.0002209; 0.0002328]         0  0.00%
     na1_e     EX_na1_e 0.0002906  [0.000

# Creating Model 

In [32]:
from cobra.io import write_sbml_model

In [33]:
with Syn6803:
    Ethanol_Off(Syn6803)
    Lactate_Off(Syn6803)
    Ethylene_Off(Syn6803)
    Isobutanol_Off(Syn6803)
    Butan_1_ol_Off(Syn6803)
    Farnesene_Off(Syn6803)
    Limonene_Off(Syn6803)
    Butanediol_Off(Syn6803)
    Syn6803.reactions.EX_glc__D_e.bounds = (-999999,0)
    Syn6803.reactions.EX_hco3_e.bounds = (-999999, 0)
    Syn6803.reactions.EX_photon_e.bounds = (-999999, 0)
    Syn6803.reactions.NDH1_3u_1.bounds = (0,0)
    Syn6803.reactions.NDH1_4pp_1.bounds = (0,0)
    write_sbml_model(Syn6803, 'iSynCJ816_STOCHIOMETRIC_for_AUTOPACMAN.xml')